<a href="https://colab.research.google.com/github/ysys143/ml2024/blob/main/finding_news_v0_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
!pip install -q langchain #언어모델을 활용한 어플리케이션 제작에 도움을 주는 프레임워크
!pip install -q -U langchain-community #랭체인에서 제공하는 추가 기능

!pip install -q langchain_openai #gemini 언어모델
!pip install -q sentence-transformers #문장을 벡터로 변환하는데 사용하는 라이브러리
!pip install -q unstructured #비정형 데이터를 처리하고 구조화된 형태로 변환하는 데 사용됨

!pip install -q tiktoken # 토큰화
!pip install -q faiss-cpu # 벡터 검색, 유사도 검색

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 1.8 MB/s eta 0:00:00


In [38]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

In [25]:
import os
import openai
import json
import requests
from datetime import datetime

from langchain import LLMChain, OpenAI, PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredURLLoader

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

from langchain.utilities import GoogleSerperAPIWrapper

In [27]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_key')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
embeddings = OpenAIEmbeddings()

<ipython-input-27-3a213f3d4230>:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [28]:
!pip install --upgrade nltk

import nltk
nltk.download('punkt')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## 시기에 맞는 keyword return

In [29]:
import pandas as pd
from datetime import datetime

# 데이터셋을 데이터프레임으로 정의
data = {
    "date": ["2022.07.08", "2022.08.27", "2023.01.21", "2023.01.28", "2023.05.21",
             "2023.06.21", "2023.07.23", "2023.08.09", "2024.01.16", "2024.07.21"],
    "group": ["NewJeans"] * 10,
    "title": ["민지 티셔츠 문구 선정성 논란", "Cookie 가사 논란", "다니엘 중국 설 표기 논란",
              "한지 관련 중국 네티즌 악플 테러", "멕시코 걸그룹 '진스' 표절 의혹",
              "X 배틀그라운드 콜라보 관련 논란", "Bubble Gum 표절 논란",
              "음악방송 중 아이폰 퍼포먼스 논란", "민지  칼국수 발언 관련 논란",
              "뉴진스도 모르는 생일카페 광주광역시 동구청"],
    "url": ["https://news.mt.co.kr/mtview.php?no=2022072810214588016",
            "https://www.mbn.co.kr/news/society/4831628",
            "https://www.etoday.co.kr/news/view/2215018",
            "https://idsn.co.kr/news/view/1065598944642006",
            "https://www.insight.co.kr/news/469214",
            "https://www.ekoreanews.co.kr/news/articleView.html?idxno=74032",
            "https://mksports.co.kr/view/2024/541018",
            "https://www.hankyung.com/society/article/2023080969067",
            "https://www.joongang.co.kr/article/25222428",
            "https://view.asiae.co.kr/article/2024072116494121787"]
}

df = pd.DataFrame(data)

# 날짜 차이를 계산하는 함수
def calculate_date_diff(row, query_date):
    event_date = datetime.strptime(row['date'], "%Y.%m.%d")
    return abs((query_date - event_date).days)

# 가장 가까운 사건 k개를 리턴하는 함수
def find_closest_events(group, query_date, k):
    # 입력된 날짜 변환
    query_date = datetime.strptime(query_date, "%Y-%m-%d")

    # 특정 그룹의 사건에 대해 날짜 차이를 계산하여 새로운 열에 추가
    df_filtered = df[df['group'] == group].copy()
    df_filtered['date_diff'] = df_filtered.apply(lambda row: calculate_date_diff(row, query_date), axis=1)

    # 날짜 차이가 적은 순으로 정렬 후 상위 k개 선택
    df_sorted = df_filtered.sort_values('date_diff').head(k)

    # 필요한 정보만 반환
    return df_sorted[['title', 'url']]

In [30]:
# 예시 질문
question_date = "2023-01-25"
group = "NewJeans"
k=3

# 가장 가까운 사건 찾기
closest_events = find_closest_events(group, question_date, k)


['https://idsn.co.kr/news/view/1065598944642006',
 'https://www.etoday.co.kr/news/view/2215018',
 'https://www.insight.co.kr/news/469214']

## 잘 맞는 아티클 찾아서 url 리턴

URL 로 받아 온 아티클의 내용 요약하기

In [31]:
def extract_content_from_urls(urls):

  loader = UnstructuredURLLoader(urls=urls)
  data = loader.load()

  text_splitter = CharacterTextSplitter(
      separator='\n',
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
  )

  docs = text_splitter.split_documents(data)
  db = FAISS.from_documents(docs, embeddings)

  return db

요약 진행하기

In [81]:
def summarizer(db, query, articles, k=3):
    # 유사도 검색을 통해 문서 가져오기
    docs = db.similarity_search(query, k=k)

    llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
    template = """
    {docs}

    이 요약문은 분석용 보고서에 첨부될 거야. 잘 작성된 보고서가 되기 위해 아래의 양식을 지켜야 해.

    가이드라인에 맞춰서 적어줘.

    1. 콘텐츠로부터 정보를 얻을 수 있고 유용해야 해.
    2. 콘텐츠가 너무 짧거나 길지 않고 144자로 제한해야 해.
    3. 콘텐츠는 일반적인 내용이 아닌, "{query}" 토픽을 잘 나타내고 있는 내용이어야 해.
    4. 콘텐츠는 읽기 쉽게 쓰여야 하고, 간결해야 해.
    5. 콘텐츠는 독자에게 영감을 줄 수 있어야 해.

    읽기 편하게 한국어로 작성해 줘. 답변에 가이드라인의 내용을 포함되지 않도록 할 것.

    SUMMARY :
    """

    prompt_template = PromptTemplate(input_variables=["docs", "query"], template=template)

    # 기사별 요약과 메타데이터를 포함한 출력
    summaries = []
    for doc in docs:
        summarize_chain = prompt_template | llm
        response = summarize_chain.invoke({"docs": doc.page_content, "query": query})
        summary = response.content.replace("\n", "")

        # 검색된 문서와 원본 기사 매칭 (문서 내용이 해당 URL을 포함하는지 확인)
        for article in articles:
            if article['title'] in doc.page_content or article['url'] in doc.page_content:
                title = article['title']
                url = article['url']
                formatted_output = f"[{title}]\n{url}\n{summary}"
                summaries.append(formatted_output)

    # 각 요약을 합쳐 최종 결과로 반환
    return "\n\n".join(summaries)

In [33]:
def create_query(group, date):

    # 문자열을 datetime 객체로 변환
    date_obj = datetime.strptime(date, '%Y-%m-%d')

    # 연도와 월 추출
    year = date_obj.year
    month = date_obj.month

    return f'{year}년 {month}월 {group}'

In [49]:
def get_chatbot_response(group, date):
    """
    Generate a response from the Gemini chatbot based on the selected data point.
    """
    try:
        #query = create_query(group, date)
        query = create_query(group, question_date)
        closest_event = find_closest_events(group, question_date, k)
        urls = closest_event['url'].to_list()
        data = extract_content_from_urls(urls)
        summaries = summarizer(db=data, query=query)

        return urls, summaries

    except Exception as e:
        print(f"An error occurred while generating the response: {str(e)}")
        return [], ""

In [64]:
# 예시 질문
group = "NewJeans"
question_date = "2023-01-25"
k=3

# 질문 생성
query = create_query(group, question_date)

# 가장 가까운 사건과 기사 찾기
closest_events = find_closest_events(group, question_date, k)
articles=closest_events.to_dict('records')
urls = closest_events['url'].to_list()

# 기사 내용을 추출한 뒤 요약
data = extract_content_from_urls(urls)
summaries = summarizer(db=data, query=query, articles=articles)


## test

In [82]:
summaries = summarizer(db=data, query=query, articles=articles)
summaries

"[멕시코 걸그룹 '진스' 표절 의혹]\nhttps://www.insight.co.kr/news/469214\n뉴진스가 멕시코 걸그룹 진스의 90년대 스타일을 표절했다는 의혹이 제기되었다. 두 그룹은 비주얼 콘셉트와 안무, 홍보 방식에서 유사점을 보이며, 이름에서도 '진스'라는 공통점을 가지고 있다. 이러한 논란은 뉴진스의 독창성을 주장하는 입장과 상반되며, 최근 하이브와의 갈등 속에서 더욱 주목받고 있다."

In [84]:
def extract_and_summarize_from_urls(urls, query, articles, k=3):
    # URL에서 텍스트 로딩 및 문서 분할
    loader = UnstructuredURLLoader(urls=urls)
    data = loader.load()

    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )

    docs = text_splitter.split_documents(data)

    # FAISS에 임베딩된 문서 저장
    db = FAISS.from_documents(docs, embeddings)

    # LLM 준비
    llm = ChatOpenAI(model="gpt-4o", temperature=0.5)
    template = """
    {docs}

    이 요약문은 분석용 보고서에 첨부될 거야. 잘 작성된 보고서가 되기 위해 아래의 양식을 지켜야 해.

    가이드라인에 맞춰서 적어줘.

    1. 콘텐츠로부터 정보를 얻을 수 있고 유용해야 해.
    2. 콘텐츠가 너무 짧거나 길지 않고 144자로 제한해야 해.
    3. 콘텐츠는 일반적인 내용이 아닌, "{query}" 토픽을 잘 나타내고 있는 내용이어야 해.
    4. 콘텐츠는 읽기 쉽게 쓰여야 하고, 간결해야 해.
    5. 콘텐츠는 독자에게 영감을 줄 수 있어야 해.

    읽기 편하게 한국어로 작성해 줘. 답변에 가이드라인의 내용을 포함되지 않도록 할 것.

    SUMMARY :
    """

    prompt_template = PromptTemplate(input_variables=["docs", "query"], template=template)

    # 각 기사별로 따로 요약 생성
    summaries = []
    for article in articles:
        article_content = None
        for doc in docs:
            if article['title'] in doc.page_content or article['url'] in doc.page_content:
                article_content = doc.page_content
                break  # 해당 기사와 일치하는 문서가 있으면 중단

        if article_content:
            summarize_chain = prompt_template | llm
            response = summarize_chain.invoke({"docs": article_content, "query": query})
            summary = response.content.replace("\n", "")
            title = article['title']
            url = article['url']
            formatted_output = f"[{title}]\n{url}\n{summary}"
            summaries.append(formatted_output)

    # 각 요약을 합쳐 최종 결과로 반환
    return "\n\n".join(summaries)

def create_query(group, date):
    # 문자열을 datetime 객체로 변환
    date_obj = datetime.strptime(date, '%Y-%m-%d')

    # 연도와 월 추출
    year = date_obj.year
    month = date_obj.month

    return f'{year}년 {month}월 {group}'

In [85]:
# 예시 질문
group = "NewJeans"
question_date = "2023-01-25"
k = 3

# 질문 생성
query = create_query(group, question_date)

# 가장 가까운 사건과 기사 찾기
closest_events = find_closest_events(group, question_date, k)
articles = closest_events.to_dict('records')
urls = closest_events['url'].to_list()

# 기사 내용 추출 및 요약
summaries = extract_and_summarize_from_urls(urls, query, articles, k)

print(summaries)


[한지 관련 중국 네티즌 악플 테러]
https://idsn.co.kr/news/view/1065598944642006
2023년 1월, NewJeans는 중국에서의 활동과 관련된 논란에 직면했습니다. 멤버들은 악플과 동북공정 논란으로 어려움을 겪었지만, 팬들의 지지로 이를 극복하고 있습니다.

[멕시코 걸그룹 '진스' 표절 의혹]
https://www.insight.co.kr/news/469214
뉴진스가 90년대 멕시코 걸그룹 진스를 표절했다는 의혹이 제기됐다. 유사한 의상과 콘셉트, 이름 등이 논란의 중심에 있다.
